In [1]:
# encoding=utf-8
import os.path as osp
import os
import copy
import matplotlib.pyplot as plt
import torch
from torch.nn import Linear
from sklearn.metrics import average_precision_score, roc_auc_score
from torch_geometric.data import TemporalData
from torch_geometric.datasets import JODIEDataset
from torch_geometric.datasets import ICEWS18
from torch_geometric.nn import TGNMemory, TransformerConv
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn.models.tgn import (LastNeighborLoader, IdentityMessage, MeanAggregator,
                                           LastAggregator)
from torch_geometric import *
from torch_geometric.utils import negative_sampling
from tqdm import tqdm
import networkx as nx
import numpy as np
import math
import copy
import re
import time
import json
import pandas as pd
from random import choice
import gc
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device='cpu'
# msg structure:      [src_node_feature,edge_attr,dst_node_feature]

# compute the best partition 
import datetime
# import community as community_louvain

import xxhash
#  Find the edge index which the edge vector is corresponding to
def tensor_find(t,x):
    t_np=t.cpu().numpy()
    idx=np.argwhere(t_np==x)
    return idx[0][0]+1


def std(t):
    t = np.array(t)
    return np.std(t)


def var(t):
    t = np.array(t)
    return np.var(t)


def mean(t):
    t = np.array(t)
    return np.mean(t)

def hashgen(l):
    """Generate a single hash value from a list. @l is a list of
    string values, which can be properties of a node/edge. This
    function returns a single hashed integer value."""
    hasher = xxhash.xxh64()
    for e in l:
        hasher.update(e)
    return hasher.intdigest()


def cal_pos_edges_loss(link_pred_ratio):
    loss=[]
    for i in link_pred_ratio:
        loss.append(criterion(i,torch.ones(1)))
    return torch.tensor(loss)

def cal_pos_edges_loss_multiclass(link_pred_ratio,labels):
    loss=[] 
    for i in range(len(link_pred_ratio)):
        loss.append(criterion(link_pred_ratio[i].reshape(1,-1),labels[i].reshape(-1)))
    return torch.tensor(loss)

/home/buchta/anaconda3/envs/kairos/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def cal_pos_edges_loss_autoencoder(decoded,msg):
    loss=[] 
    for i in range(len(decoded)):
        loss.append(criterion(decoded[i].reshape(-1),msg[i].reshape(-1)))
    return torch.tensor(loss)

In [3]:
from datetime import datetime, timezone
import time
import pytz
from time import mktime
from datetime import datetime
import time
def ns_time_to_datetime(ns):
    """
    :param ns: int nano timestamp
    :return: datetime   format: 2013-10-10 23:40:00.000000000
    """
    dt = datetime.fromtimestamp(int(ns) // 1000000000)
    s = dt.strftime('%Y-%m-%d %H:%M:%S')
    s += '.' + str(int(int(ns) % 1000000000)).zfill(9)
    return s

def ns_time_to_datetime_US(ns):
    """
    :param ns: int nano timestamp
    :return: datetime   format: 2013-10-10 23:40:00.000000000
    """
    tz = pytz.timezone('US/Eastern')
    dt = pytz.datetime.datetime.fromtimestamp(int(ns) // 1000000000, tz)
    s = dt.strftime('%Y-%m-%d %H:%M:%S')
    s += '.' + str(int(int(ns) % 1000000000)).zfill(9)
    return s

def time_to_datetime_US(s):
    """
    :param ns: int nano timestamp
    :return: datetime   format: 2013-10-10 23:40:00
    """
    tz = pytz.timezone('US/Eastern')
    dt = pytz.datetime.datetime.fromtimestamp(int(s), tz)
    s = dt.strftime('%Y-%m-%d %H:%M:%S')

    return s

def datetime_to_ns_time(date):
    """
    :param date: str   format: %Y-%m-%d %H:%M:%S   e.g. 2013-10-10 23:40:00
    :return: nano timestamp
    """
    timeArray = time.strptime(date, "%Y-%m-%d %H:%M:%S")
    timeStamp = int(time.mktime(timeArray))
    timeStamp = timeStamp * 1000000000
    return timeStamp

def datetime_to_ns_time_US(date):
    """
    :param date: str   format: %Y-%m-%d %H:%M:%S   e.g. 2013-10-10 23:40:00
    :return: nano timestamp
    """
    tz = pytz.timezone('US/Eastern')
    timeArray = time.strptime(date, "%Y-%m-%d %H:%M:%S")
    dt = datetime.fromtimestamp(mktime(timeArray))
    timestamp = tz.localize(dt)
    timestamp = timestamp.timestamp()
    timeStamp = timestamp * 1000000000
    return int(timeStamp)

def datetime_to_timestamp_US(date):
    """
    :param date: str   format: %Y-%m-%d %H:%M:%S   e.g. 2013-10-10 23:40:00
    :return: nano timestamp
    """
    tz = pytz.timezone('US/Eastern')
    timeArray = time.strptime(date, "%Y-%m-%d %H:%M:%S")
    dt = datetime.fromtimestamp(mktime(timeArray))
    timestamp = tz.localize(dt)
    timestamp = timestamp.timestamp()
    timeStamp = timestamp
    return int(timeStamp)

In [4]:
rel2id={1: 'EVENT_CONNECT',
 'EVENT_CONNECT': 1,
 2: 'EVENT_EXECUTE',
 'EVENT_EXECUTE': 2,
 3: 'EVENT_OPEN',
 'EVENT_OPEN': 3,
 4: 'EVENT_READ',
 'EVENT_READ': 4,
 5: 'EVENT_RECVFROM',
 'EVENT_RECVFROM': 5,
 6: 'EVENT_RECVMSG',
 'EVENT_RECVMSG': 6,
 7: 'EVENT_SENDMSG',
 'EVENT_SENDMSG': 7,
 8: 'EVENT_SENDTO',
 'EVENT_SENDTO': 8,
 9: 'EVENT_WRITE',
 'EVENT_WRITE': 9}

In [5]:
import psycopg2

from psycopg2 import extras as ex
connect = psycopg2.connect(database = 'tc_theia_dataset_db',
                           host = '141.71.31.196',
                           user = 'kairos',
                           password = 'lolroflxd',
                           port = '54321'
                          )

cur = connect.cursor()

In [6]:
# Constructing the map for nodeid to msg
sql="select * from node2id ORDER BY index_id;"
cur.execute(sql)
rows = cur.fetchall()

nodeid2msg={}  # nodeid => msg and node hash => nodeid
for i in rows:
    nodeid2msg[i[0]]=i[-1]
    nodeid2msg[i[-1]]={i[1]:i[2]}  #0-828297

## Load data

In [7]:
graph_4_3=torch.load("./train_graph/graph_4_3.TemporalData.simple").to(device=device)
graph_4_4=torch.load("./train_graph/graph_4_4.TemporalData.simple").to(device=device)
graph_4_5=torch.load("./train_graph/graph_4_5.TemporalData.simple").to(device=device)
graph_4_9=torch.load("./train_graph/graph_4_9.TemporalData.simple").to(device=device)

graph_4_10=torch.load("./train_graph/graph_4_10.TemporalData.simple").to(device=device)
graph_4_11=torch.load("./train_graph/graph_4_11.TemporalData.simple").to(device=device)
graph_4_12=torch.load("./train_graph/graph_4_12.TemporalData.simple").to(device=device)
graph_4_13=torch.load("./train_graph/graph_4_13.TemporalData.simple").to(device=device)
train_data=graph_4_10

# GNN

In [8]:
[
    graph_4_3.num_nodes,
    graph_4_4.num_nodes,
    graph_4_5.num_nodes,
    graph_4_9.num_nodes,
    graph_4_10.num_nodes,
    graph_4_11.num_nodes,
    graph_4_12.num_nodes,
    graph_4_13.num_nodes,
]

[828311, 828304, 828187, 746145, 815985, 826308, 826255, 826255]

In [9]:
max_node_num = 828398 
min_dst_idx, max_dst_idx = 0, max_node_num
neighbor_loader = LastNeighborLoader(max_node_num, size=20, device=device)

class GraphAttentionEmbedding(torch.nn.Module):
    def __init__(self, in_channels, out_channels, msg_dim, time_enc):
        super(GraphAttentionEmbedding, self).__init__()
        self.time_enc = time_enc
        edge_dim = msg_dim + time_enc.out_channels
        self.conv = TransformerConv(in_channels, out_channels, heads=8,
                                    dropout=0.0, edge_dim=edge_dim)
        self.conv2 = TransformerConv(in_channels*8, out_channels,heads=1, concat=False,
                             dropout=0.0, edge_dim=edge_dim)

    def forward(self, x, last_update, edge_index, t, msg):
        last_update.to(device)
        x = x.to(device)
        t = t.to(device)
        rel_t = last_update[edge_index[0]] - t
        rel_t_enc = self.time_enc(rel_t.to(x.dtype))
        edge_attr = torch.cat([rel_t_enc, msg], dim=-1)
        x = F.relu(self.conv(x, edge_index, edge_attr))
        x = F.relu(self.conv2(x, edge_index, edge_attr))
        return x

class LinkPredictor(torch.nn.Module):
    def __init__(self, in_channels):
        super(LinkPredictor, self).__init__()
        self.lin_src = Linear(in_channels, in_channels*2)
        self.lin_dst = Linear(in_channels, in_channels*2)
        
        self.lin_seq = nn.Sequential(
            
            Linear(in_channels*4, in_channels*8),
            torch.nn.Dropout(0.5),
            nn.Tanh(),
            Linear(in_channels*8, in_channels*2),
            torch.nn.Dropout(0.5),
            nn.Tanh(),
            Linear(in_channels*2, int(in_channels//2)),
            torch.nn.Dropout(0.5),
            nn.Tanh(),
            Linear(int(in_channels//2), train_data.msg.shape[1]-32)                   
        )
        
    def forward(self, z_src, z_dst):
        h = torch.cat([self.lin_src(z_src) , self.lin_dst(z_dst)],dim=-1)      
         
        h = self.lin_seq (h)
        
        return h


memory_dim = time_dim = embedding_dim = 100

memory = TGNMemory(
    max_node_num,
    train_data.msg.size(-1),
    memory_dim,
    time_dim,
    message_module=IdentityMessage(train_data.msg.size(-1), memory_dim, time_dim),
    aggregator_module=LastAggregator(),
).to(device)

gnn = GraphAttentionEmbedding(
    in_channels=memory_dim,
    out_channels=embedding_dim,
    msg_dim=train_data.msg.size(-1),
    time_enc=memory.time_enc,
).to(device)

link_pred = LinkPredictor(in_channels=embedding_dim).to(device)

optimizer = torch.optim.Adam(
    set(memory.parameters()) | set(gnn.parameters())
    | set(link_pred.parameters()), lr=0.00005, eps=1e-08,weight_decay=0.01)


# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
criterion = nn.CrossEntropyLoss()

# Helper vector to map global node indices to local ones.
assoc = torch.empty(max_node_num, dtype=torch.long, device=device)


saved_nodes=set()

In [10]:
BATCH=1024
def train(train_data):

    
    memory.train()
    gnn.train()
    link_pred.train()

    memory.reset_state()  # Start with a fresh memory.
    neighbor_loader.reset_state()  # Start with an empty graph.
    saved_nodes=set()

    total_loss = 0
    total_batchs=len(train_data.t)//BATCH +1
    batch_index=0
#     print("train_before_stage_data:",train_data)
    for batch in train_data.seq_batches(batch_size=BATCH):
        start = time.perf_counter()
        optimizer.zero_grad()

        src, pos_dst, t, msg = batch.src, batch.dst, batch.t, batch.msg        
        
        n_id = torch.cat([src, pos_dst]).unique()
#         n_id = torch.cat([src, pos_dst, neg_src, neg_dst]).unique()
        n_id, edge_index, e_id = neighbor_loader(n_id)
        assoc[n_id] = torch.arange(n_id.size(0), device=device)

        # Get updated memory of all nodes involved in the computation.
        z, last_update = memory(n_id)
        
#         print(f"{n_id=}")
#         print(f"{edge_index=}")
#         print(f"{train_data.msg[e_id]=}")
      
        z = gnn(z, last_update, edge_index, train_data.t[e_id], train_data.msg[e_id])
        
        pos_out = link_pred(z[assoc[src]], z[assoc[pos_dst]])       

        y_pred = torch.cat([pos_out], dim=0)
        
#         y_true = torch.cat([torch.zeros(pos_out.size(0),1),torch.ones(neg_out.size(0),1)], dim=0)
        y_true=[]
        for m in msg:
            l=tensor_find(m[16:-16],1)-1
            y_true.append(l)           
          
        y_true = torch.tensor(y_true)
        y_true=y_true.reshape(-1).to(torch.long).to(device=device)
        
        loss = criterion(y_pred, y_true)
        

#         loss = criterion(pos_out, torch.ones_like(pos_out))
#         loss += criterion(neg_out, torch.zeros_like(neg_out))

        # Update memory and neighbor loader with ground-truth state.
        memory.update_state(src, pos_dst, t, msg)
        neighbor_loader.insert(src, pos_dst)
        
#         for i in range(len(src)):
#             saved_nodes.add(int(src[i]))
#             saved_nodes.add(int(pos_dst[i]))

        loss.backward()
        optimizer.step()
        memory.detach()
#         print(z.shape)
        total_loss += float(loss) * batch.num_events
        batch_index+=1
        end = time.perf_counter()
#         print(f"current epoch process:{(batch_index/total_batchs):.4f}%   cost time:{(end-start):.2f}")
#     print("trained_stage_data:",train_data)
    return total_loss / train_data.num_events


In [11]:
print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Allocated: 7.5 GB
Cached:    7.6 GB


In [12]:
print("hello")

hello


## Start to train

In [13]:
train_graphs=[
    graph_4_3,
    graph_4_4, 
    graph_4_5,
#     graph_4_9
]

for epoch in tqdm(range(1, 51)):
    for g in train_graphs:
        loss = train(g)
        print(f'  Epoch: {epoch:02d}, Loss: {loss:.4f}')
#     scheduler.step()

model=[memory,gnn, link_pred,neighbor_loader]
os.system("mkdir -p ./models/")
torch.save(model,"./models/model_saved_emb100_BATCH_1024_LastAggregator_multiclass_without_neg_edge.pt")

  0%|                                                                                                                                                                   | 0/50 [00:00<?, ?it/s]

  Epoch: 01, Loss: 0.6764
  Epoch: 01, Loss: 0.3115


  2%|███                                                                                                                                                     | 1/50 [11:33<9:26:29, 693.67s/it]

  Epoch: 01, Loss: 0.3597
  Epoch: 02, Loss: 0.2630
  Epoch: 02, Loss: 0.2743


  4%|██████                                                                                                                                                  | 2/50 [23:05<9:14:04, 692.60s/it]

...

  Epoch: 48, Loss: 0.2594
  Epoch: 49, Loss: 0.2232
  Epoch: 49, Loss: 0.2349


 98%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉   | 49/50 [9:20:18<10:52, 652.99s/it]

  Epoch: 49, Loss: 0.3028
  Epoch: 50, Loss: 0.2235
  Epoch: 50, Loss: 0.2451


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [9:31:11<00:00, 685.43s/it]

  Epoch: 50, Loss: 0.2760


# Test

In [14]:
import time 

@torch.no_grad()
def test_day_new(inference_data,path):
    if os.path.exists(path):
        pass
    else:
        os.mkdir(path)
    
#     m=torch.load("model_saved_emb100.pt")
#     memory,gnn, link_pred,neighbor_loader=m
    memory.eval()
    gnn.eval()
    link_pred.eval()
    
    memory.reset_state()  # Start with a fresh memory.  
    neighbor_loader.reset_state()  # Start with an empty graph.
    
    time_with_loss={}
    total_loss = 0    
    edge_list=[]
    
    unique_nodes=torch.tensor([]).to(device=device)
    total_edges=0
    
#     test_memory=copy.deepcopy(memory)   
#     test_gnn=copy.deepcopy(gnn)   
#     test_link_pred=copy.deepcopy(link_pred) 
#     test_neighbor_loader=copy.deepcopy(neighbor_loader)



    start_time=inference_data.t[0]
    event_count=0
    
    pos_o=[]
    
    loss_list=[]
    
#     print("before merge:",train_data)

#     nique_node_count=len(torch.cat([train_data.src,train_data.dst]).unique())

    print("after merge:",inference_data)
    
    # Record the running time to evaluate the performance
    start = time.perf_counter()

    for batch in inference_data.seq_batches(batch_size=BATCH):
        
        src, pos_dst, t, msg = batch.src, batch.dst, batch.t, batch.msg
        unique_nodes=torch.cat([unique_nodes,src,pos_dst]).unique()
        total_edges+=BATCH
        
       
        n_id = torch.cat([src, pos_dst]).unique()       
        n_id, edge_index, e_id = neighbor_loader(n_id)
        assoc[n_id] = torch.arange(n_id.size(0), device=device)

        z, last_update = memory(n_id)
        z = gnn(z, last_update, edge_index, inference_data.t[e_id], inference_data.msg[e_id])

        pos_out = link_pred(z[assoc[src]], z[assoc[pos_dst]])
        
        pos_o.append(pos_out)
        y_pred = torch.cat([pos_out], dim=0)
#         y_true = torch.cat(
#             [torch.ones(pos_out.size(0))], dim=0).to(torch.long)     
#         y_true=y_true.reshape(-1).to(torch.long)

        y_true=[]
        for m in msg:
            l=tensor_find(m[16:-16],1)-1
            y_true.append(l) 
        y_true = torch.tensor(y_true)
        y_true=y_true.reshape(-1).to(torch.long).to(device=device)

        # Only consider which edge hasn't been correctly predicted.
        # For benign graphs, the behaviors patterns are similar and therefore their losses are small
        # For anoamlous behaviors, some behaviors might not be seen before, so the probability of predicting those edges are low. Thus their losses are high.
        loss = criterion(y_pred, y_true)

        total_loss += float(loss) * batch.num_events
     
        
        # update the edges in the batch to the memory and neighbor_loader
        memory.update_state(src, pos_dst, t, msg)
        neighbor_loader.insert(src, pos_dst)
        
        # compute the loss for each edge
        each_edge_loss= cal_pos_edges_loss_multiclass(pos_out,y_true)
        
        for i in range(len(pos_out)):
            srcnode=int(src[i])
            dstnode=int(pos_dst[i])  
            
            srcmsg=str(nodeid2msg[srcnode]) 
            dstmsg=str(nodeid2msg[dstnode])
            t_var=int(t[i])
            edgeindex=tensor_find(msg[i][16:-16],1)    
            edge_type=rel2id[edgeindex]
            loss=each_edge_loss[i]    

            temp_dic={}
            temp_dic['loss']=float(loss)
            temp_dic['srcnode']=srcnode
            temp_dic['dstnode']=dstnode
            temp_dic['srcmsg']=srcmsg
            temp_dic['dstmsg']=dstmsg
            temp_dic['edge_type']=edge_type
            temp_dic['time']=t_var
            

#             if "netflow" in srcmsg or "netflow" in dstmsg:
#                 temp_dic['loss']=0
            edge_list.append(temp_dic)
        
        event_count+=len(batch.src)
        if t[-1]>start_time+60000000000*15:
            # Here is a checkpoint, which records all edge losses in the current time window
#             loss=total_loss/event_count
            time_interval=ns_time_to_datetime_US(start_time)+"~"+ns_time_to_datetime_US(t[-1])

            end = time.perf_counter()
            time_with_loss[time_interval]={'loss':loss,
                                
                                          'nodes_count':len(unique_nodes),
                                          'total_edges':total_edges,
                                          'costed_time':(end-start)}
            
            
            log=open(path+"/"+time_interval+".txt",'w')
            
            for e in edge_list: 
#                 temp_key=e['srcmsg']+e['dstmsg']+e['edge_type']
#                 if temp_key in train_edge_set:      
# #                     e['loss']=(e['loss']-train_edge_set[temp_key]) if e['loss']>=train_edge_set[temp_key] else 0  
# #                     e['loss']=abs(e['loss']-train_edge_set[temp_key])
                    
#                     e['modified']=True
#                 else:
#                     e['modified']=False
                loss+=e['loss']

            loss=loss/event_count   
            print(f'Time: {time_interval}, Loss: {loss:.4f}, Nodes_count: {len(unique_nodes)}, Cost Time: {(end-start):.2f}s')
            edge_list = sorted(edge_list, key=lambda x:x['loss'],reverse=True)   # Rank the results based on edge loss
            for e in edge_list: 
                log.write(str(e))
                log.write("\n") 
            event_count=0
            total_loss=0
            loss=0
            start_time=t[-1]
            log.close()
            edge_list.clear()
            
 
    return time_with_loss


# Test 4-9 ~ 4-12

In [15]:
model=torch.load("./models/model_saved_emb100_BATCH_1024_LastAggregator_multiclass_without_neg_edge.pt",map_location=device)
memory,gnn, link_pred,neighbor_loader=model
ans_4_3=test_day_new(graph_4_3,"graph_4_3")

after merge: TemporalData(dst=[8230837], msg=[8230837, 41], src=[8230837], t=[8230837])
Time: 2018-04-03 10:02:14.348882872~2018-04-03 10:17:22.845633004, Loss: 1.2002, Nodes_count: 2021, Cost Time: 3.68s
Time: 2018-04-03 10:17:22.845633004~2018-04-03 10:32:52.909926097, Loss: 2.1182, Nodes_count: 5070, Cost Time: 13.65s
Time: 2018-04-03 10:32:52.909926097~2018-04-03 10:47:54.761593566, Loss: 1.2590, Nodes_count: 9219, Cost Time: 30.47s
Time: 2018-04-03 10:47:54.761593566~2018-04-03 11:03:00.878438338, Loss: 0.5627, Nodes_count: 13173, Cost Time: 47.01s
Time: 2018-04-03 11:03:00.878438338~2018-04-03 11:18:31.001956600, Loss: 0.2615, Nodes_count: 17464, Cost Time: 68.71s
Time: 2018-04-03 11:18:31.001956600~2018-04-03 11:33:31.167232690, Loss: 0.3987, Nodes_count: 20766, Cost Time: 94.70s
Time: 2018-04-03 11:33:31.167232690~2018-04-03 11:48:41.419633970, Loss: 0.3228, Nodes_count: 24304, Cost Time: 117.77s
Time: 2018-04-03 11:48:41.419633970~2018-04-03 12:03:45.030247497, Loss: 0.2420, N

In [16]:
model=torch.load("./models/model_saved_emb100_BATCH_1024_LastAggregator_multiclass_without_neg_edge.pt",map_location=device)
memory,gnn, link_pred,neighbor_loader=model
ans_4_4=test_day_new(graph_4_4,"graph_4_4")

after merge: TemporalData(dst=[4930304], msg=[4930304, 41], src=[4930304], t=[4930304])
Time: 2018-04-04 00:00:00.001798512~2018-04-04 00:15:02.197293670, Loss: 0.5815, Nodes_count: 363, Cost Time: 1.48s
Time: 2018-04-04 00:15:02.197293670~2018-04-04 00:30:28.978098397, Loss: 0.7388, Nodes_count: 658, Cost Time: 3.04s
Time: 2018-04-04 00:30:28.978098397~2018-04-04 00:45:32.001547272, Loss: 2.9602, Nodes_count: 987, Cost Time: 4.60s
Time: 2018-04-04 00:45:32.001547272~2018-04-04 01:01:12.511397075, Loss: 4.6379, Nodes_count: 1228, Cost Time: 6.14s
Time: 2018-04-04 01:01:12.511397075~2018-04-04 01:16:50.001162895, Loss: 4.8345, Nodes_count: 1488, Cost Time: 7.69s
Time: 2018-04-04 01:16:50.001162895~2018-04-04 01:32:05.001431940, Loss: 4.7370, Nodes_count: 1751, Cost Time: 9.24s
Time: 2018-04-04 01:32:05.001431940~2018-04-04 01:47:27.933058106, Loss: 4.7859, Nodes_count: 2030, Cost Time: 10.79s
Time: 2018-04-04 01:47:27.933058106~2018-04-04 02:02:58.001409515, Loss: 4.8036, Nodes_count: 2

In [17]:
model=torch.load("./models/model_saved_emb100_BATCH_1024_LastAggregator_multiclass_without_neg_edge.pt",map_location=device)
memory,gnn, link_pred,neighbor_loader=model
ans_4_5=test_day_new(graph_4_5,"graph_4_5")

after merge: TemporalData(dst=[1489011], msg=[1489011, 41], src=[1489011], t=[1489011])
Time: 2018-04-05 00:00:00.001766694~2018-04-05 00:16:00.002476684, Loss: 0.6076, Nodes_count: 346, Cost Time: 1.37s
Time: 2018-04-05 00:16:00.002476684~2018-04-05 00:31:52.143639968, Loss: 0.3960, Nodes_count: 650, Cost Time: 2.80s
Time: 2018-04-05 00:31:52.143639968~2018-04-05 00:47:52.002321096, Loss: 0.3894, Nodes_count: 937, Cost Time: 4.23s
Time: 2018-04-05 00:47:52.002321096~2018-04-05 01:03:35.001510674, Loss: 0.3576, Nodes_count: 1217, Cost Time: 5.66s
Time: 2018-04-05 01:03:35.001510674~2018-04-05 01:19:09.002324977, Loss: 0.3636, Nodes_count: 1489, Cost Time: 7.08s
Time: 2018-04-05 01:19:09.002324977~2018-04-05 01:35:25.002311363, Loss: 0.3539, Nodes_count: 1746, Cost Time: 8.51s
Time: 2018-04-05 01:35:25.002311363~2018-04-05 01:51:29.002313221, Loss: 0.3738, Nodes_count: 2011, Cost Time: 9.93s
Time: 2018-04-05 01:51:29.002313221~2018-04-05 02:07:14.002084758, Loss: 0.3744, Nodes_count: 22

In [18]:
model=torch.load("./models/model_saved_emb100_BATCH_1024_LastAggregator_multiclass_without_neg_edge.pt",map_location=device)
memory,gnn, link_pred,neighbor_loader=model
ans_4_9=test_day_new(graph_4_9,"graph_4_9")

after merge: TemporalData(dst=[685635], msg=[685635, 41], src=[685635], t=[685635])
Time: 2018-04-09 08:46:55.004764124~2018-04-09 09:03:31.001287346, Loss: 2.3671, Nodes_count: 27, Cost Time: 0.12s
Time: 2018-04-09 09:03:31.001287346~2018-04-09 09:20:23.001295997, Loss: 0.2942, Nodes_count: 67, Cost Time: 0.85s
Time: 2018-04-09 09:20:23.001295997~2018-04-09 09:36:02.001305059, Loss: 0.3195, Nodes_count: 100, Cost Time: 1.37s
Time: 2018-04-09 09:36:02.001305059~2018-04-09 09:51:31.358485271, Loss: 0.7597, Nodes_count: 5561, Cost Time: 4.91s
Time: 2018-04-09 09:51:31.358485271~2018-04-09 10:06:48.907525397, Loss: 2.0088, Nodes_count: 6439, Cost Time: 10.70s
Time: 2018-04-09 10:06:48.907525397~2018-04-09 10:23:05.001376520, Loss: 1.9359, Nodes_count: 13241, Cost Time: 20.57s
Time: 2018-04-09 10:23:05.001376520~2018-04-09 10:39:33.001366978, Loss: 4.1025, Nodes_count: 13594, Cost Time: 23.39s
Time: 2018-04-09 10:39:33.001366978~2018-04-09 10:58:03.657948688, Loss: 3.8822, Nodes_count: 138

In [19]:
model=torch.load("./models/model_saved_emb100_BATCH_1024_LastAggregator_multiclass_without_neg_edge.pt",map_location=device)
memory,gnn, link_pred,neighbor_loader=model
ans_4_10=test_day_new(graph_4_10,"graph_4_10")

after merge: TemporalData(dst=[6274151], msg=[6274151, 41], src=[6274151], t=[6274151])
Time: 2018-04-10 12:44:33.449564893~2018-04-10 13:00:02.700560774, Loss: 0.6469, Nodes_count: 701, Cost Time: 0.65s
Time: 2018-04-10 13:00:02.700560774~2018-04-10 13:16:13.551944728, Loss: 0.2036, Nodes_count: 1373, Cost Time: 14.80s
Time: 2018-04-10 13:16:13.551944728~2018-04-10 13:31:14.548738409, Loss: 0.4577, Nodes_count: 6803, Cost Time: 30.05s
Time: 2018-04-10 13:31:14.548738409~2018-04-10 13:46:36.161065223, Loss: 0.9377, Nodes_count: 11267, Cost Time: 58.44s
Time: 2018-04-10 13:46:36.161065223~2018-04-10 14:02:17.001271389, Loss: 5.5124, Nodes_count: 11336, Cost Time: 60.79s
Time: 2018-04-10 14:02:17.001271389~2018-04-10 14:17:34.001373488, Loss: 2.7976, Nodes_count: 11950, Cost Time: 64.47s
Time: 2018-04-10 14:17:34.001373488~2018-04-10 14:33:18.350772859, Loss: 6.9140, Nodes_count: 13208, Cost Time: 362.09s
Time: 2018-04-10 14:33:18.350772859~2018-04-10 14:48:47.320442910, Loss: 2.0297, No

In [20]:
model=torch.load("./models/model_saved_emb100_BATCH_1024_LastAggregator_multiclass_without_neg_edge.pt",map_location=device)
memory,gnn, link_pred,neighbor_loader=model
ans_4_11=test_day_new(graph_4_11,"graph_4_11")

after merge: TemporalData(dst=[7285220], msg=[7285220, 41], src=[7285220], t=[7285220])
Time: 2018-04-11 00:00:00.001151329~2018-04-11 00:16:06.001274623, Loss: 0.5046, Nodes_count: 117, Cost Time: 1.23s
Time: 2018-04-11 00:16:06.001274623~2018-04-11 00:32:08.001169192, Loss: 0.3877, Nodes_count: 257, Cost Time: 2.51s
Time: 2018-04-11 00:32:08.001169192~2018-04-11 00:48:31.001594545, Loss: 0.8374, Nodes_count: 336, Cost Time: 3.79s
Time: 2018-04-11 00:48:31.001594545~2018-04-11 01:04:53.001888013, Loss: 2.2972, Nodes_count: 413, Cost Time: 5.06s
Time: 2018-04-11 01:04:53.001888013~2018-04-11 01:20:59.002307553, Loss: 4.9573, Nodes_count: 492, Cost Time: 6.34s
Time: 2018-04-11 01:20:59.002307553~2018-04-11 01:37:06.826246565, Loss: 5.5935, Nodes_count: 575, Cost Time: 7.61s
Time: 2018-04-11 01:37:06.826246565~2018-04-11 01:53:33.001963923, Loss: 5.9176, Nodes_count: 649, Cost Time: 8.89s
Time: 2018-04-11 01:53:33.001963923~2018-04-11 02:10:00.001194307, Loss: 5.9350, Nodes_count: 724, C

In [21]:
model=torch.load("./models/model_saved_emb100_BATCH_1024_LastAggregator_multiclass_without_neg_edge.pt",map_location=device)
memory,gnn, link_pred,neighbor_loader=model
ans_4_12=test_day_new(graph_4_12,"graph_4_12")

after merge: TemporalData(dst=[7024937], msg=[7024937, 41], src=[7024937], t=[7024937])
Time: 2018-04-12 00:00:00.001773757~2018-04-12 00:15:26.001647081, Loss: 0.6014, Nodes_count: 120, Cost Time: 0.99s
Time: 2018-04-12 00:15:26.001647081~2018-04-12 00:30:58.002002412, Loss: 0.4111, Nodes_count: 252, Cost Time: 2.03s
Time: 2018-04-12 00:30:58.002002412~2018-04-12 00:46:27.001835122, Loss: 0.2844, Nodes_count: 330, Cost Time: 3.06s
Time: 2018-04-12 00:46:27.001835122~2018-04-12 01:02:05.698953597, Loss: 0.3301, Nodes_count: 402, Cost Time: 4.09s
Time: 2018-04-12 01:02:05.698953597~2018-04-12 01:17:27.002277528, Loss: 0.3579, Nodes_count: 477, Cost Time: 5.13s
Time: 2018-04-12 01:17:27.002277528~2018-04-12 01:32:48.001732365, Loss: 0.4378, Nodes_count: 594, Cost Time: 6.16s
Time: 2018-04-12 01:32:48.001732365~2018-04-12 01:48:20.001593370, Loss: 0.4403, Nodes_count: 671, Cost Time: 7.19s
Time: 2018-04-12 01:48:20.001593370~2018-04-12 02:04:08.001895394, Loss: 0.4375, Nodes_count: 742, C

## Compute anomlous score and Initialize the node IDF

In [22]:
def cal_train_IDF(find_str,file_list):
    include_count=0
    for f_path in (file_list):
        f=open(f_path)
        if find_str in f.read():
            include_count+=1             
    IDF=math.log(len(file_list)/(include_count+1))
    return IDF


def cal_IDF(find_str,file_path,file_list):
    file_list=os.listdir(file_path)
    include_count=0
    different_neighbor=set()
    for f_path in (file_list):
        f=open(file_path+f_path)
        if find_str in f.read():
            include_count+=1
#         add=True
#         for line in f:
            
#             if find_str in line:
# #                 print(line)
#                 if add:
#                     include_count+=1
#                     add=False
#                 l=line.strip()
#                 jdata=eval(l)
#                 different_neighbor.add(jdata['srcmsg'])
#                 different_neighbor.add(jdata['dstmsg'])
                
                
    IDF=math.log(len(file_list)/(include_count+1))
    
    return IDF,1

def cal_IDF_by_file_in_mem(find_str,file_list):

    include_count=0
    different_neighbor=set()
    for f_path in (file_list):
        f=open(file_path+f_path)
        if find_str in f.read():
            include_count+=1
#         add=True
#         for line in f:
            
#             if find_str in line:
# #                 print(line)
#                 if add:
#                     include_count+=1
#                     add=False
#                 l=line.strip()
#                 jdata=eval(l)
#                 different_neighbor.add(jdata['srcmsg'])
#                 different_neighbor.add(jdata['dstmsg'])
                
                
    IDF=math.log(len(file_list)/(include_count+1))
    
    return IDF,1

def cal_redundant(find_str,edge_list):
    
    different_neighbor=set()
    for e in edge_list:
        if find_str in str(e):
            different_neighbor.add(e[0])
            different_neighbor.add(e[1])
    return len(different_neighbor)-2

def cal_anomaly_loss(loss_list,edge_list,file_path):
    
    if len(loss_list)!=len(edge_list):
        print("error!")
        return 0
    count=0
    loss_sum=0
    loss_std=std(loss_list)
    loss_mean=mean(loss_list)
    edge_set=set()
    node_set=set()
    node2redundant={}
    
    thr=loss_mean+1.5*loss_std

    print("thr:",thr)
  
    for i in range(len(loss_list)):
        if loss_list[i]>thr:
            count+=1
            src_node=edge_list[i][0]
            dst_node=edge_list[i][1]
          
            loss_sum+=loss_list[i]
    
            node_set.add(src_node)
            node_set.add(dst_node)
            edge_set.add(edge_list[i][0]+edge_list[i][1])
    return count, loss_sum/(count+0.00000000001),node_set,edge_set
#     return count, count/len(loss_list)

In [23]:

node_IDF={}
node_set=set()

file_list=[]

file_path="graph_4_3/"
file_l=os.listdir("graph_4_3/")
for i in file_l:
    file_list.append(file_path+i)

file_path="graph_4_4/"
file_l=os.listdir("graph_4_4/")
for i in file_l:
    file_list.append(file_path+i)

file_path="graph_4_5/"
file_l=os.listdir("graph_4_5/")
for i in file_l:
    file_list.append(file_path+i)

    
node_set = {}
for f_path in tqdm(file_list):
    f=open(f_path)
    for line in f:
        l=line.strip()
        jdata=eval(l)
        if jdata['loss']>0:
            if 'netflow' not in str(jdata['srcmsg']) or True:
                if str(jdata['srcmsg']) not in node_set.keys():
                    node_set[str(jdata['srcmsg'])] = set([f_path])
                else:
                    node_set[str(jdata['srcmsg'])].add(f_path)
            if 'netflow' not in str(jdata['dstmsg']) or True:
                if str(jdata['dstmsg']) not in node_set.keys():
                    node_set[str(jdata['dstmsg'])] = set([f_path])
                else:
                    node_set[str(jdata['dstmsg'])].add(f_path)
for n in node_set:
    include_count = len(node_set[n])   
    IDF=math.log(len(file_list)/(include_count+1))
    node_IDF[n] = IDF 


torch.save(node_IDF,"node_IDF_4_3-5")
print("IDF weight calculate complete!")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 194/194 [03:57<00:00,  1.22s/it]


IDF weight calculate complete!


In [24]:
node_IDF={}
node_set=set()

file_list=[]

file_path="graph_4_10/"
file_l=os.listdir("graph_4_10/")
for i in file_l:
    file_list.append(file_path+i)


node_set = {}
for f_path in tqdm(file_list):
    f=open(f_path)
    for line in f:
        l=line.strip()
        jdata=eval(l)
        if jdata['loss']>0:
            if 'netflow' not in str(jdata['srcmsg']) or True:
                if str(jdata['srcmsg']) not in node_set.keys():
                    node_set[str(jdata['srcmsg'])] = set([f_path])
                else:
                    node_set[str(jdata['srcmsg'])].add(f_path)
            if 'netflow' not in str(jdata['dstmsg']) or True:
                if str(jdata['dstmsg']) not in node_set.keys():
                    node_set[str(jdata['dstmsg'])] = set([f_path])
                else:
                    node_set[str(jdata['dstmsg'])].add(f_path)
for n in node_set:
    include_count = len(node_set[n])   
    IDF=math.log(len(file_list)/(include_count+1))
    node_IDF[n] = IDF 
    

torch.save(node_IDF,"node_IDF_4_10")
print("IDF weight calculate complete!")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 43/43 [01:42<00:00,  2.39s/it]

IDF weight calculate complete!


In [25]:
node_IDF={}
node_set=set()

file_list=[]

file_path="graph_4_11/"
file_l=os.listdir("graph_4_11/")
for i in file_l:
    file_list.append(file_path+i)


node_set = {}
for f_path in tqdm(file_list):
    f=open(f_path)
    for line in f:
        l=line.strip()
        jdata=eval(l)
        if jdata['loss']>0:
            if 'netflow' not in str(jdata['srcmsg']) or True:
                if str(jdata['srcmsg']) not in node_set.keys():
                    node_set[str(jdata['srcmsg'])] = set([f_path])
                else:
                    node_set[str(jdata['srcmsg'])].add(f_path)
            if 'netflow' not in str(jdata['dstmsg']) or True:
                if str(jdata['dstmsg']) not in node_set.keys():
                    node_set[str(jdata['dstmsg'])] = set([f_path])
                else:
                    node_set[str(jdata['dstmsg'])].add(f_path)
for n in node_set:
    include_count = len(node_set[n])   
    IDF=math.log(len(file_list)/(include_count+1))
    node_IDF[n] = IDF 


torch.save(node_IDF,"node_IDF_4_11")
print("IDF weight calculate complete!")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 91/91 [01:58<00:00,  1.30s/it]


IDF weight calculate complete!


In [26]:
node_IDF={}
node_set=set()

file_list=[]

file_path="graph_4_12/"
file_l=os.listdir("graph_4_12/")
for i in file_l:
    file_list.append(file_path+i)


node_set = {}
for f_path in tqdm(file_list):
    f=open(f_path)
    for line in f:
        l=line.strip()
        jdata=eval(l)
        if jdata['loss']>0:
            if 'netflow' not in str(jdata['srcmsg']) or True:
                if str(jdata['srcmsg']) not in node_set.keys():
                    node_set[str(jdata['srcmsg'])] = set([f_path])
                else:
                    node_set[str(jdata['srcmsg'])].add(f_path)
            if 'netflow' not in str(jdata['dstmsg']) or True:
                if str(jdata['dstmsg']) not in node_set.keys():
                    node_set[str(jdata['dstmsg'])] = set([f_path])
                else:
                    node_set[str(jdata['dstmsg'])].add(f_path)
for n in node_set:
    include_count = len(node_set[n])   
    IDF=math.log(len(file_list)/(include_count+1))
    node_IDF[n] = IDF 
    

torch.save(node_IDF,"node_IDF_4_12")
print("IDF weight calculate complete!")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 93/93 [01:54<00:00,  1.23s/it]


IDF weight calculate complete!


# Construct the relations between time windows

In [27]:
# 4-10,11
def is_include_key_word_bak(s):
    keywords=[
         'netflow',
        'null',
        '/dev/pts',
        'salt-minion.log',
        '675',
        'usr',
         'proc',
        '/.cache/mozilla/',
        'tmp',
        'thunderbird',
        '/bin/',
        '/sbin/sysctl',
        '/data/replay_logdb/',
        '/home/admin/eraseme',
        
        '/stat',
        
      ]
    flag=False
    for i in keywords:
        if i in s:
            flag=True
    return flag


def cal_set_rel_bak(s1,s2,file_list):
    new_s=s1 & s2
    count=0
    for i in new_s:
#     jdata=json.loads(i)
        if is_include_key_word_bak(i) is not True:
            if i in node_IDF.keys():
                IDF=node_IDF[i]
            else:
                IDF=math.log(len(file_list)/(1))         

            if (IDF)>math.log(len(file_list)*0.9/(1))  :
                print("node:",i," IDF:",IDF)
                count+=1
    return count

In [28]:
# 4-12

# def is_include_key_word_bak(s):
#     keywords=[
#          'netflow',        
#         '/dev/pts',
#         'salt-minion.log',
#         'null',
#         'usr',
#          'proc',
#         'firefox',
#         'tmp',
#         'thunderbird',
#         'bin/',
#         '/data/replay_logdb',
#         '/stat',
#         '/boot',
#         'qt-opensource-linux-x64',
#         '/eraseme',
#         '675',
        
# #       
# #         'etc',  
# #         'cdrom', 
# #         'shm'
#       ]
#     flag=False
#     for i in keywords:
#         if i in s:
#             flag=True
#     return flag

# def cal_set_rel_bak(s1,s2,file_list):
#     new_s=s1 & s2
#     count=0
#     for i in new_s:
# #     jdata=json.loads(i)
#         if is_include_key_word_bak(i) is not True:
#             if i in node_IDF.keys():
#                 IDF=node_IDF[i]
#             else:
#                 IDF=math.log(len(file_list)/(1))         

#             if (IDF)>math.log(len(file_list)*0.9/(1))  :
#                 print("node:",i," IDF:",IDF)
#                 count+=1
#     return count


def is_include_key_word(s):
    keywords=[
         'netflow',        
        '/dev/pts',
        'salt-minion.log',
        'null',
        'usr',
         'proc',
        'firefox',
        'tmp',
        'thunderbird',
        'bin/',
        '/data/replay_logdb',
        '/stat',
        '/boot',
        'qt-opensource-linux-x64',
        '/eraseme',
        '675',
      ]
    flag=False
    for i in keywords:
        if i in s:
            flag=True
    return flag


file_list=[]

file_path="graph_4_12/"
file_l=os.listdir("graph_4_12/")
for i in file_l:
    file_list.append(file_path+i)
    


def cal_set_rel(s1,s2,file_list, file_list_4_3_5):
    IDF3 = node_IDF_3
    new_s=s1 & s2
    count=0
    for i in new_s:
#     jdata=json.loads(i)
       if is_include_key_word(i) is not True:
        
#         'netflow' not in i
#         and 'usr' not in i and 'var' not in i
            if i in node_IDF.keys():
                IDF=node_IDF[i]
            else:
                IDF=math.log(len(file_list)/(1))
                
            if i in node_IDF_3.keys():
                IDF3=node_IDF_3[i]
            else:
                IDF3=math.log(len(file_list_4_3_5)/(1))    
            
#             print(IDF)
            if (IDF+IDF3)>5 :
                print("node:",i," IDF:",IDF)
                count+=1
    return count

# label generation

In [29]:
labels={}

    
filelist = os.listdir("graph_4_10/")
for f in filelist:
    labels["graph_4_10/"+f]=0

filelist = os.listdir("graph_4_11/")
for f in filelist:
    labels["graph_4_11/"+f]=0
    
filelist = os.listdir("graph_4_12/")
for f in filelist:
    labels["graph_4_12/"+f]=0


In [30]:
sorted(labels)

['graph_4_10/2018-04-10 12:44:33.449564893~2018-04-10 13:00:02.700560774.txt',
 'graph_4_10/2018-04-10 13:00:02.700560774~2018-04-10 13:16:13.551944728.txt',
 'graph_4_10/2018-04-10 13:16:13.551944728~2018-04-10 13:31:14.548738409.txt',
 'graph_4_10/2018-04-10 13:31:14.548738409~2018-04-10 13:46:36.161065223.txt',
 'graph_4_10/2018-04-10 13:46:36.161065223~2018-04-10 14:02:17.001271389.txt',
 'graph_4_10/2018-04-10 14:02:17.001271389~2018-04-10 14:17:34.001373488.txt',
 'graph_4_10/2018-04-10 14:17:34.001373488~2018-04-10 14:33:18.350772859.txt',
 'graph_4_10/2018-04-10 14:33:18.350772859~2018-04-10 14:48:47.320442910.txt',
 'graph_4_10/2018-04-10 14:48:47.320442910~2018-04-10 15:03:54.307022037.txt',
 'graph_4_10/2018-04-10 15:03:54.307022037~2018-04-10 15:19:25.001773315.txt',
 'graph_4_10/2018-04-10 15:19:25.001773315~2018-04-10 15:36:13.002273705.txt',
 'graph_4_10/2018-04-10 15:36:13.002273705~2018-04-10 15:51:24.614585595.txt',
 'graph_4_10/2018-04-10 15:51:24.614585595~2018-04-1

In [31]:
attack_list=[
    
'graph_4_10/2018-04-10 13:31:14.548738409~2018-04-10 13:46:36.161065223.txt',
'graph_4_10/2018-04-10 14:02:17.001271389~2018-04-10 14:17:34.001373488.txt',
'graph_4_10/2018-04-10 14:17:34.001373488~2018-04-10 14:33:18.350772859.txt',
'graph_4_10/2018-04-10 14:33:18.350772859~2018-04-10 14:48:47.320442910.txt',
'graph_4_10/2018-04-10 14:48:47.320442910~2018-04-10 15:03:54.307022037.txt', 
 
'graph_4_12/2018-04-12 12:39:06.592684498~2018-04-12 12:54:44.001888457.txt',
'graph_4_12/2018-04-12 12:54:44.001888457~2018-04-12 13:09:55.026832462.txt',
'graph_4_12/2018-04-12 13:09:55.026832462~2018-04-12 13:25:06.588370709.txt',
'graph_4_12/2018-04-12 13:25:06.588370709~2018-04-12 13:40:07.178206094.txt',
]

for i in attack_list:
    labels[i]=1

    

# Anomaly Detection

## 4-9

In [32]:
# node_IDF_3=torch.load("node_IDF_4_3")
node_IDF=torch.load("node_IDF_4_3-5")
file_list=[]

file_path="graph_4_9/"
file_l=os.listdir("graph_4_9/")
for i in file_l:
    file_list.append(file_path+i)
    
    
    
# node_IDF_410=torch.load("node_IDF_4_10")
# node_IDF=torch.load("node_IDF_4_12")
y_data_4_10=[]
df_list_4_10=[]
# node_set_list=[]
history_list=[]
tw_que=[]
his_tw={}
current_tw={}



file_path_list=[]


file_path="graph_4_9/"
file_l=os.listdir("graph_4_9/")
for i in file_l:
    file_path_list.append(file_path+i)

# file_path="graph_4_12/"
# file_l=os.listdir("graph_4_12/")
# for i in file_l:
#     file_path_list.append(file_path+i)


index_count=0
for f_path in sorted(file_path_list):
    f=open(f_path)
    edge_loss_list=[]
    edge_list=[]
    print('index_count:',index_count)
    
    for line in f:
        l=line.strip()
        jdata=eval(l)
        edge_loss_list.append(jdata['loss'])
        edge_list.append([str(jdata['srcmsg']),str(jdata['dstmsg'])])
    df_list_4_10.append(pd.DataFrame(edge_loss_list))
    count,loss_avg,node_set,edge_set=cal_anomaly_loss(edge_loss_list,edge_list,"graph_4_10_without_neg_edge/")
    current_tw={}
    current_tw['name']=f_path
    current_tw['loss']=loss_avg
    current_tw['index']=index_count
    current_tw['nodeset']=node_set

    added_que_flag=False
    for hq in history_list:
        for his_tw in hq:
            if cal_set_rel_bak(current_tw['nodeset'],his_tw['nodeset'],file_list)!=0 and current_tw['name']!=his_tw['name']:
                print("history queue:",his_tw['name'])
                hq.append(copy.deepcopy(current_tw))
                added_que_flag=True
                break
        if added_que_flag:
            break
    if added_que_flag is False:
        temp_hq=[copy.deepcopy(current_tw)]
        history_list.append(temp_hq)
  
    index_count+=1
#     node_set_list.append(node_set)
    print( f_path,"  ",loss_avg," count:",count," percentage:",count/len(edge_list)," node count:",len(node_set)," edge count:",len(edge_set))
#     y_data_4_10.append([loss_avg,labels_4_10[f_path],f_path])

index_count: 0
thr: 4.333107333517309
graph_4_9/2018-04-09 08:46:55.004764124~2018-04-09 09:03:31.001287346.txt    4.744216918943957  count: 35  percentage: 0.0341796875  node count: 2  edge count: 1
index_count: 1
thr: 1.1452938830380068
graph_4_9/2018-04-09 09:03:31.001287346~2018-04-09 09:20:23.001295997.txt    4.096786728065545  count: 89  percentage: 0.014485677083333334  node count: 25  edge count: 25
index_count: 2
thr: 1.527041761347741
graph_4_9/2018-04-09 09:20:23.001295997~2018-04-09 09:36:02.001305059.txt    4.893667713979107  count: 89  percentage: 0.021728515625  node count: 12  edge count: 14
index_count: 3
thr: 2.2394586064155524
graph_4_9/2018-04-09 09:36:02.001305059~2018-04-09 09:51:31.358485271.txt    4.145739035415888  count: 1427  percentage: 0.05161313657407408  node count: 90  edge count: 104
index_count: 4
thr: 5.891414138148244
graph_4_9/2018-04-09 09:51:31.358485271~2018-04-09 10:06:48.907525397.txt    7.19428587069515  count: 7391  percentage: 0.160394965277

In [33]:

# pred_label={}

# files = os.listdir("graph_4_9")
# for f in files:
#     pred_label["graph_4_9/"+f]=0

# files = os.listdir("graph_4_10")
# for f in files:
#     pred_label["graph_4_10/"+f]=0

# files = os.listdir("graph_4_11")
# for f in files:
#     pred_label["graph_4_11/"+f]=0

# files = os.listdir("graph_4_12")
# for f in files:
#     pred_label["graph_4_12/"+f]=0


for hl in history_list:
    loss_count=0
    for hq in hl:
        if loss_count==0:
            loss_count=(loss_count+1)*(hq['loss']+1)
        else:
            loss_count=(loss_count)*(hq['loss']+1)

    if loss_count>10:
#     if loss_count>50:
        name_list=[]
        for i in hl:
            name_list.append(i['name'])
            print(i['name'])
#         print(name_list)
        print(loss_count)

## 4-10

In [34]:
# node_IDF_3=torch.load("node_IDF_4_3")
node_IDF=torch.load("node_IDF_4_3-5")
file_list=[]

file_path="graph_4_10/"
file_l=os.listdir("graph_4_10/")
for i in file_l:
    file_list.append(file_path+i)
    
    
    
# node_IDF_410=torch.load("node_IDF_4_10")
# node_IDF=torch.load("node_IDF_4_12")
y_data_4_10=[]
df_list_4_10=[]
# node_set_list=[]
history_list=[]
tw_que=[]
his_tw={}
current_tw={}



file_path_list=[]


file_path="graph_4_10/"
file_l=os.listdir("graph_4_10/")
for i in file_l:
    file_path_list.append(file_path+i)

# file_path="graph_4_12/"
# file_l=os.listdir("graph_4_12/")
# for i in file_l:
#     file_path_list.append(file_path+i)


index_count=0
for f_path in sorted(file_path_list):
    f=open(f_path)
    edge_loss_list=[]
    edge_list=[]
    print('index_count:',index_count)
    
    for line in f:
        l=line.strip()
        jdata=eval(l)
        edge_loss_list.append(jdata['loss'])
        edge_list.append([str(jdata['srcmsg']),str(jdata['dstmsg'])])
    df_list_4_10.append(pd.DataFrame(edge_loss_list))
    count,loss_avg,node_set,edge_set=cal_anomaly_loss(edge_loss_list,edge_list,"graph_4_10_without_neg_edge/")
    current_tw={}
    current_tw['name']=f_path
    current_tw['loss']=loss_avg
    current_tw['index']=index_count
    current_tw['nodeset']=node_set

    added_que_flag=False
    for hq in history_list:
        for his_tw in hq:
            if cal_set_rel_bak(current_tw['nodeset'],his_tw['nodeset'],file_list)!=0 and current_tw['name']!=his_tw['name']:
                print("history queue:",his_tw['name'])
                hq.append(copy.deepcopy(current_tw))
                added_que_flag=True
                break
        if added_que_flag:
            break
    if added_que_flag is False:
        temp_hq=[copy.deepcopy(current_tw)]
        history_list.append(temp_hq)
  
    index_count+=1
#     node_set_list.append(node_set)
    print( f_path,"  ",loss_avg," count:",count," percentage:",count/len(edge_list)," node count:",len(node_set)," edge count:",len(edge_set))
#     y_data_4_10.append([loss_avg,labels_4_10[f_path],f_path])

index_count: 0
thr: 1.9873775298080156
graph_4_10/2018-04-10 12:44:33.449564893~2018-04-10 13:00:02.700560774.txt    3.134815188835503  count: 359  percentage: 0.0701171875  node count: 50  edge count: 52
index_count: 1
thr: 1.2453563433222592
graph_4_10/2018-04-10 13:00:02.700560774~2018-04-10 13:16:13.551944728.txt    2.6209736324944783  count: 7521  percentage: 0.06277544070512821  node count: 158  edge count: 200
index_count: 2
thr: 2.2907515291846265
graph_4_10/2018-04-10 13:16:13.551944728~2018-04-10 13:31:14.548738409.txt    4.498610347367828  count: 7369  percentage: 0.06257642663043478  node count: 955  edge count: 1514
index_count: 3
thr: 3.6782980559657816
graph_4_10/2018-04-10 13:31:14.548738409~2018-04-10 13:46:36.161065223.txt    6.117775564474104  count: 20306  percentage: 0.08892411715246637  node count: 306  edge count: 435
index_count: 4
thr: 10.030441142039683
graph_4_10/2018-04-10 13:46:36.161065223~2018-04-10 14:02:17.001271389.txt    0.0  count: 0  percentage: 0.0

In [35]:

pred_label={}

# files = os.listdir("graph_4_9")
# for f in files:
#     pred_label["graph_4_9/"+f]=0

files = os.listdir("graph_4_10")
for f in files:
    pred_label["graph_4_10/"+f]=0

files = os.listdir("graph_4_11")
for f in files:
    pred_label["graph_4_11/"+f]=0

files = os.listdir("graph_4_12")
for f in files:
    pred_label["graph_4_12/"+f]=0


for hl in history_list:
    loss_count=0
    for hq in hl:
        if loss_count==0:
            loss_count=(loss_count+1)*(hq['loss']+1)
        else:
            loss_count=(loss_count)*(hq['loss']+1)

    if loss_count>14:
#     if loss_count>50:
        name_list=[]
        for i in hl:
            name_list.append(i['name'])
            print(i['name'])
#         print(name_list)
        for i in name_list:
            pred_label[i]=1
        print(loss_count)

## 4-11

In [36]:
# node_IDF_3=torch.load("node_IDF_4_3")
node_IDF=torch.load("node_IDF_4_3-5")
file_list=[]

file_path="graph_4_11/"
file_l=os.listdir("graph_4_11/")
for i in file_l:
    file_list.append(file_path+i)
    
    
    
# node_IDF_410=torch.load("node_IDF_4_10")
# node_IDF=torch.load("node_IDF_4_12")
y_data_4_10=[]
df_list_4_10=[]
# node_set_list=[]
history_list=[]
tw_que=[]
his_tw={}
current_tw={}



file_path_list=[]


file_path="graph_4_11/"
file_l=os.listdir("graph_4_11/")
for i in file_l:
    file_path_list.append(file_path+i)

# file_path="graph_4_12/"
# file_l=os.listdir("graph_4_12/")
# for i in file_l:
#     file_path_list.append(file_path+i)


index_count=0
for f_path in sorted(file_path_list):
    f=open(f_path)
    edge_loss_list=[]
    edge_list=[]
    print('index_count:',index_count)
    
    for line in f:
        l=line.strip()
        jdata=eval(l)
        edge_loss_list.append(jdata['loss'])
        edge_list.append([str(jdata['srcmsg']),str(jdata['dstmsg'])])
    df_list_4_10.append(pd.DataFrame(edge_loss_list))
    count,loss_avg,node_set,edge_set=cal_anomaly_loss(edge_loss_list,edge_list,"graph_4_11_without_neg_edge/")
    current_tw={}
    current_tw['name']=f_path
    current_tw['loss']=loss_avg
    current_tw['index']=index_count
    current_tw['nodeset']=node_set

    added_que_flag=False
    for hq in history_list:
        for his_tw in hq:
            if cal_set_rel_bak(current_tw['nodeset'],his_tw['nodeset'],file_list)!=0 and current_tw['name']!=his_tw['name']:
                print("history queue:",his_tw['name'])
                hq.append(copy.deepcopy(current_tw))
                added_que_flag=True
                break
        if added_que_flag:
            break
    if added_que_flag is False:
        temp_hq=[copy.deepcopy(current_tw)]
        history_list.append(temp_hq)
  
    index_count+=1
#     node_set_list.append(node_set)
    print( f_path,"  ",loss_avg," count:",count," percentage:",count/len(edge_list)," node count:",len(node_set)," edge count:",len(edge_set))
#     y_data_4_10.append([loss_avg,labels_4_10[f_path],f_path])

index_count: 0
thr: 2.0386619074766372
graph_4_11/2018-04-11 00:00:00.001151329~2018-04-11 00:16:06.001274623.txt    3.4046089742217607  count: 974  percentage: 0.0951171875  node count: 36  edge count: 40
index_count: 1
thr: 1.1867972728528304
graph_4_11/2018-04-11 00:16:06.001274623~2018-04-11 00:32:08.001169192.txt    2.1365745458069614  count: 519  percentage: 0.05068359375  node count: 38  edge count: 39
index_count: 2
thr: 1.816301447065461
graph_4_11/2018-04-11 00:32:08.001169192~2018-04-11 00:48:31.001594545.txt    2.8210524318386105  count: 538  percentage: 0.0525390625  node count: 31  edge count: 38
index_count: 3
thr: 4.503158041412749
graph_4_11/2018-04-11 00:48:31.001594545~2018-04-11 01:04:53.001888013.txt    4.844501662885019  count: 945  percentage: 0.09228515625  node count: 13  edge count: 12
index_count: 4
thr: 8.62445716616936
graph_4_11/2018-04-11 01:04:53.001888013~2018-04-11 01:20:59.002307553.txt    9.108321030929591  count: 18  percentage: 0.0017578125  node c

In [37]:
for hl in history_list:
    loss_count=0
    for hq in hl:
        if loss_count==0:
            loss_count=(loss_count+1)*(hq['loss']+1)
        else:
            loss_count=(loss_count)*(hq['loss']+1)
    name_list=[]
    if loss_count>25:
#     if loss_count>50:
        name_list=[]
        for i in hl:
            name_list.append(i['name'])
        print(name_list)
#         for i in name_list:
#             pred_label[i]=1
        print(loss_count)

## 4-12

In [38]:
file_list_3_5=[]

file_path="graph_4_3/"
file_l=os.listdir("graph_4_3/")
for i in file_l:
    file_list_3_5.append(file_path+i)

file_path="graph_4_4/"
file_l=os.listdir("graph_4_4/")
for i in file_l:
    file_list_3_5.append(file_path+i)

file_path="graph_4_5/"
file_l=os.listdir("graph_4_5/")
for i in file_l:
    file_list_3_5.append(file_path+i)
    
len(file_list_3_5)

194

In [39]:
len(file_list)

91

In [40]:
node_IDF=torch.load("node_IDF_4_12") 
node_IDF_3=torch.load("node_IDF_4_3-5")
# node_IDF=torch.load("node_IDF_4_3")
file_list=[]

file_path="graph_4_12/"
file_l=os.listdir("graph_4_12/")
for i in file_l:
    file_list.append(file_path+i)
    
# the variable names doesn't change the results.   
y_data_4_10=[]
df_list_4_10=[]
history_list=[]
tw_que=[]
his_tw={}
current_tw={}



file_path_list=[]


file_path="graph_4_12/"
file_l=os.listdir("graph_4_12/")
for i in file_l:
    file_path_list.append(file_path+i)


index_count=0
for f_path in sorted(file_path_list):
    f=open(f_path)
    edge_loss_list=[]
    edge_list=[]
    print('index_count:',index_count)
    
    for line in f:
        l=line.strip()
        jdata=eval(l)
        edge_loss_list.append(jdata['loss'])
        edge_list.append([str(jdata['srcmsg']),str(jdata['dstmsg'])])
    df_list_4_10.append(pd.DataFrame(edge_loss_list))
    count,loss_avg,node_set,edge_set=cal_anomaly_loss(edge_loss_list,edge_list,"graph_4_12_without_neg_edge/")
    current_tw={}
    current_tw['name']=f_path
    current_tw['loss']=loss_avg
    current_tw['index']=index_count
    current_tw['nodeset']=node_set

    added_que_flag=False
    for hq in history_list:
        for his_tw in hq:
            cal_re = cal_set_rel(current_tw['nodeset'],his_tw['nodeset'],file_list, file_list_3_5)
            if cal_re != 0 and current_tw['name']!=his_tw['name']:
#             if cal_set_rel_bak(current_tw['nodeset'],his_tw['nodeset'],file_l)!=0 and current_tw['name']!=his_tw['name']:
                hq.append(copy.deepcopy(current_tw))
                added_que_flag=True
                break
        if added_que_flag:
            break
    if added_que_flag is False:
        temp_hq=[copy.deepcopy(current_tw)]
        history_list.append(temp_hq)
    index_count+=1
    print( f_path,"  ",loss_avg," count:",count," percentage:",count/len(edge_list)," node count:",len(node_set)," edge count:",len(edge_set))
    
    
    
#     for i in history_list:
#         print(len(i))
#         if len(i) >= 2:
#             for tw in i:
#                 print(tw['name'])
#     input()
    
    

index_count: 0
thr: 2.316749470191067
graph_4_12/2018-04-12 00:00:00.001773757~2018-04-12 00:15:26.001647081.txt    3.4387840541397305  count: 984  percentage: 0.1201171875  node count: 41  edge count: 44
index_count: 1
thr: 1.4765361136167985
graph_4_12/2018-04-12 00:15:26.001647081~2018-04-12 00:30:58.002002412.txt    2.7757993924094557  count: 479  percentage: 0.0584716796875  node count: 43  edge count: 44
index_count: 2
thr: 1.482830602911302
graph_4_12/2018-04-12 00:30:58.002002412~2018-04-12 00:46:27.001835122.txt    3.0481515053808677  count: 523  percentage: 0.0638427734375  node count: 34  edge count: 38
index_count: 3
thr: 1.7658733055389781
graph_4_12/2018-04-12 00:46:27.001835122~2018-04-12 01:02:05.698953597.txt    3.4674714539006692  count: 599  percentage: 0.0731201171875  node count: 32  edge count: 34
index_count: 4
thr: 2.0189172992846336
graph_4_12/2018-04-12 01:02:05.698953597~2018-04-12 01:17:27.002277528.txt    3.897136765367784  count: 646  percentage: 0.0788574

In [41]:
for hl in history_list:
    loss_count=0
    for hq in hl:
        if loss_count==0:
            loss_count=(loss_count+1)*(hq['loss']+1)
        else:
            loss_count=(loss_count)*(hq['loss']+1)
    name_list=[]
    if loss_count>20:
#     if loss_count>50:
        name_list=[]
        for i in hl:
            name_list.append(i['name'])
        print(name_list)
        for i in name_list:
            pred_label[i]=1
        print(loss_count)

['graph_4_12/2018-04-12 09:19:03.668714727~2018-04-12 09:34:16.330657912.txt', 'graph_4_12/2018-04-12 12:39:06.592684498~2018-04-12 12:54:44.001888457.txt', 'graph_4_12/2018-04-12 12:54:44.001888457~2018-04-12 13:09:55.026832462.txt', 'graph_4_12/2018-04-12 13:25:06.588370709~2018-04-12 13:40:07.178206094.txt']
337.92062038408966


In [42]:
from sklearn.metrics import average_precision_score, roc_auc_score

from sklearn.metrics import roc_auc_score
import torch
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix

def plot_thr():
    np.seterr(invalid='ignore')
    step=0.01
    thr_list=torch.arange(-5,5,step)
    
    

    precision_list=[]
    recall_list=[]
    fscore_list=[]
    accuracy_list=[]
    auc_val_list=[]
    for thr in thr_list:
        threshold=thr
        y_prediction=[]
        for i in y_test_scores:
            if i >threshold:
                y_prediction.append(1)
            else:
                y_prediction.append(0)
        precision,recall,fscore,accuracy,auc_val=classifier_evaluation(y_test, y_prediction)   
        precision_list.append(float(precision))
        recall_list.append(float(recall))
        fscore_list.append(float(fscore))
        accuracy_list.append(float(accuracy))
        auc_val_list.append(float(auc_val))

    max_fscore=max(fscore_list)
    max_fscore_index=fscore_list.index(max_fscore)
    print(max_fscore_index)
    print("max threshold:",thr_list[max_fscore_index])
    print('precision:',precision_list[max_fscore_index])
    print('recall:',recall_list[max_fscore_index])
    print('fscore:',fscore_list[max_fscore_index])
    print('accuracy:',accuracy_list[max_fscore_index])    
    print('auc:',auc_val_list[max_fscore_index])
    
         
#     precision_list=torch.tensor(precision_list)   
#     recall_list=torch.tensor(recall_list)   
#     fscore_list=torch.tensor(fscore_list)   
#     accuracy_list=torch.tensor(accuracy_list)   
#     auc_val_list=torch.tensor(auc_val_list)   

    


    
    # plt.scatter(attack_x, attack_y, s=20, c='r', label='Attack graph',marker='*')
    # plt.scatter(bengin_x, bengin_y, s=20, c='g', label='Bengin graph',marker='1')
    # plt.scatter(bengin_x, bengin_y, s=20, c='g', label='Bengin graph',marker='1')

    plt.plot(thr_list,precision_list,color='red',label='precision',linewidth=2.0,linestyle='-')
    plt.plot(thr_list,recall_list,color='orange',label='recall',linewidth=2.0,linestyle='solid')
    plt.plot(thr_list,fscore_list,color='y',label='F-score',linewidth=2.0,linestyle='dashed')
    plt.plot(thr_list,accuracy_list,color='g',label='accuracy',linewidth=2.0,linestyle='dashdot')
    plt.plot(thr_list,auc_val_list,color='b',label='auc_val',linewidth=2.0,linestyle='dotted')
    # '-', '--', '-.', ':', 'None', ' ', '', 'solid', 'dashed', 'dashdot', 'dotted'


    # plt.scatter(turnovers, graph_loss, c=color)
    plt.xlabel("Threshold", fontdict={'size': 16})
    plt.ylabel("Rate", fontdict={'size': 16})
    plt.title("Different evaluation Indicators by varying threshold value", fontdict={'size': 12})
    plt.legend(loc='best', fontsize=12, markerscale=0.5)
    plt.show()

def classifier_evaluation(y_test, y_test_pred):
    # groundtruth, pred_value
    tn, fp, fn, tp =confusion_matrix(y_test, y_test_pred).ravel()
#     tn+=100
#     print(clf_name," : ")
    print('tn:',tn)
    print('fp:',fp)
    print('fn:',fn)
    print('tp:',tp)
    precision=tp/(tp+fp)
    recall=tp/(tp+fn)
    accuracy=(tp+tn)/(tp+tn+fp+fn)
    fscore=2*(precision*recall)/(precision+recall)    
    auc_val=roc_auc_score(y_test, y_test_pred)
    print("precision:",precision)
    print("recall:",recall)
    print("fscore:",fscore)
    print("accuracy:",accuracy)
    print("auc_val:",auc_val)
    return precision,recall,fscore,accuracy,auc_val

def minmax(data):
    min_val=min(data)
    max_val=max(data)
    ans=[]
    for i in data:
        ans.append((i-min_val)/(max_val-min_val))
    return ans



In [43]:
y=[]
y_pred=[]
for i in labels:
    y.append(labels[i])
    y_pred.append(pred_label[i])

In [44]:
classifier_evaluation(y,y_pred)

tn: 217
fp: 1
fn: 6
tp: 3
precision: 0.75
recall: 0.3333333333333333
fscore: 0.46153846153846156
accuracy: 0.9691629955947136
auc_val: 0.6643730886850152


(0.75,
 0.3333333333333333,
 0.46153846153846156,
 0.9691629955947136,
 0.6643730886850152)

# Count the attack edges numbers

In [45]:
def keyword_hit(line):
    attack_nodes=[
            '/home/admin/clean',
            '/dev/glx_alsa_675',
            '/home/admin/profile',
#             '/var/log/mail',  
            '/tmp/memtrace.so',
            '/var/log/xdev',
             '/var/log/wdev',
            'gtcache',
#             'firefox',
        '161.116.88.72',
        '146.153.68.151',
        '145.199.103.57',
        '61.130.69.232',
        '5.214.163.155',
        '104.228.117.212',
        '141.43.176.203',
        '7.149.198.40',
        '5.214.163.155',
        '149.52.198.23',
        ]
    flag=False
    for i in attack_nodes:
        if i in line:
            flag=True
            break
    return flag



files=[]
temp_file=[
        '2018-04-10 13:31:14.548738409~2018-04-10 13:46:36.161065223.txt',
        '2018-04-10 14:02:17.001271389~2018-04-10 14:17:34.001373488.txt',
        '2018-04-10 14:17:34.001373488~2018-04-10 14:33:18.350772859.txt',
        '2018-04-10 14:33:18.350772859~2018-04-10 14:48:47.320442910.txt',
        '2018-04-10 14:48:47.320442910~2018-04-10 15:03:54.307022037.txt',
]
for f in temp_file:
    files.append("./graph_4_10/"+f)
    
    
temp_file=[
         '2018-04-12 12:39:06.592684498~2018-04-12 12:54:44.001888457.txt',
        '2018-04-12 12:54:44.001888457~2018-04-12 13:09:55.026832462.txt',
        '2018-04-12 13:09:55.026832462~2018-04-12 13:25:06.588370709.txt',
        '2018-04-12 13:25:06.588370709~2018-04-12 13:40:07.178206094.txt',
]    
for f in temp_file:
    files.append("./graph_4_12/"+f)
    
attack_edge_count=0
for fpath in tqdm(files):
    f=open(fpath)
    for line in f:
        if keyword_hit(line):
            attack_edge_count+=1
print(attack_edge_count)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:04<00:00,  1.85it/s]

3039


# Visualization

In [46]:
import os

from graphviz import Digraph
import networkx as nx
import datetime
import community.community_louvain as community_louvain
from tqdm import tqdm




# Some common path abstraction for visualization
replace_dic = {
        '/run/shm/':'/run/shm/*',
        #     '/home/admin/.cache/mozilla/firefox/pe11scpa.default/cache2/entries/':'/home/admin/.cache/mozilla/firefox/pe11scpa.default/cache2/entries/*',
        '/home/admin/.cache/mozilla/firefox/':'/home/admin/.cache/mozilla/firefox/*',
        '/home/admin/.mozilla/firefox':'/home/admin/.mozilla/firefox*',
        '/data/replay_logdb/':'/data/replay_logdb/*',
        '/home/admin/.local/share/applications/':'/home/admin/.local/share/applications/*',
        '/usr/share/applications/':'/usr/share/applications/*',
        '/lib/x86_64-linux-gnu/':'/lib/x86_64-linux-gnu/*',
        '/proc/':'/proc/*',
        '/stat':'*/stat',
        '/etc/bash_completion.d/':'/etc/bash_completion.d/*',
        '/usr/bin/python2.7':'/usr/bin/python2.7/*',
        '/usr/lib/python2.7':'/usr/lib/python2.7/*',
}


def replace_path_name(path_name):
    for i in replace_dic:
        if i in path_name:
            return replace_dic[i]
    return path_name


# Users should manually put the detected anomalous time windows here
attack_list = [
        'graph_4_10/2018-04-10 13:31:14.548738409~2018-04-10 13:46:36.161065223.txt',
        'graph_4_10/2018-04-10 14:02:17.001271389~2018-04-10 14:17:34.001373488.txt',
        'graph_4_10/2018-04-10 14:17:34.001373488~2018-04-10 14:33:18.350772859.txt',
        'graph_4_10/2018-04-10 14:33:18.350772859~2018-04-10 14:48:47.320442910.txt',
        'graph_4_10/2018-04-10 14:48:47.320442910~2018-04-10 15:03:54.307022037.txt',
    
        'graph_4_12/2018-04-12 12:39:06.592684498~2018-04-12 12:54:44.001888457.txt', 
        'graph_4_12/2018-04-12 12:54:44.001888457~2018-04-12 13:09:55.026832462.txt', 
        'graph_4_12/2018-04-12 13:09:55.026832462~2018-04-12 13:25:06.588370709.txt', 
        'graph_4_12/2018-04-12 13:25:06.588370709~2018-04-12 13:40:07.178206094.txt'
]

original_edges_count = 0
graphs = []
gg = nx.DiGraph()
count = 0
for path in tqdm(attack_list):
    if ".txt" in path:
        line_count = 0
        node_set = set()
        tempg = nx.DiGraph()
        f = open(path, "r")
        edge_list = []
        for line in f:
            count += 1
            l = line.strip()
            jdata = eval(l)
            edge_list.append(jdata)

        edge_list = sorted(edge_list, key=lambda x: x['loss'], reverse=True)
        original_edges_count += len(edge_list)

        loss_list = []
        for i in edge_list:
            loss_list.append(i['loss'])
        loss_mean = mean(loss_list)
        loss_std = std(loss_list)
        print(loss_mean)
        print(loss_std)
        thr = loss_mean + 1.5 * loss_std
        print("thr:", thr)
        for e in edge_list:
            if e['loss'] > thr:
                tempg.add_edge(str(hashgen(replace_path_name(e['srcmsg']))),
                               str(hashgen(replace_path_name(e['dstmsg']))))
                gg.add_edge(str(hashgen(replace_path_name(e['srcmsg']))), str(hashgen(replace_path_name(e['dstmsg']))),
                            loss=e['loss'], srcmsg=e['srcmsg'], dstmsg=e['dstmsg'], edge_type=e['edge_type'],
                            time=e['time'])


partition = community_louvain.best_partition(gg.to_undirected())

# Generate the candidate subgraphs based on community discovery results
communities = {}
max_partition = 0
for i in partition:
    if partition[i] > max_partition:
        max_partition = partition[i]
for i in range(max_partition + 1):
    communities[i] = nx.DiGraph()
for e in gg.edges:
    communities[partition[e[0]]].add_edge(e[0], e[1])
    communities[partition[e[1]]].add_edge(e[0], e[1])


# Define the attack nodes. They are **only be used to plot the colors of attack nodes and edges**.
# They won't change the detection results.
def attack_edge_flag(msg):
    attack_nodes = [
        '/home/admin/clean',
        '/dev/glx_alsa_675',
        '/home/admin/profile',
        '/var/log/xdev',
        '/etc/passwd',
        '161.116.88.72',
        '146.153.68.151',
        '/var/log/mail',
        '/tmp/memtrace.so',
        #         '/tmp',
        '/var/log/xdev',
        '/var/log/wdev',
        'gtcache',
        'firefox',
        #         '/var/log',
    ]
    flag = False
    for i in attack_nodes:
        if i in str(msg):
            flag = True
    return flag


# Plot and render candidate subgraph
os.system(f"mkdir -p ./graph_visual/")
graph_index = 0
for c in communities:
    dot = Digraph(name="MyPicture", comment="the test", format="pdf")
    dot.graph_attr['rankdir'] = 'LR'

    for e in communities[c].edges:
        try:
            temp_edge = gg.edges[e]
            srcnode = e['srcnode']
            dstnode = e['dstnode']
        except:
            pass

        if True:
            # source node
            if "'subject': '" in temp_edge['srcmsg']:
                src_shape = 'box'
            elif "'file': '" in temp_edge['srcmsg']:
                src_shape = 'oval'
            elif "'netflow': '" in temp_edge['srcmsg']:
                src_shape = 'diamond'
            if attack_edge_flag(temp_edge['srcmsg']):
                src_node_color = 'red'
            else:
                src_node_color = 'blue'
            dot.node(name=str(hashgen(replace_path_name(temp_edge['srcmsg']))), label=str(
                replace_path_name(temp_edge['srcmsg']) + str(
                    partition[str(hashgen(replace_path_name(temp_edge['srcmsg'])))])), color=src_node_color,
                     shape=src_shape)

            # destination node
            if "'subject': '" in temp_edge['dstmsg']:
                dst_shape = 'box'
            elif "'file': '" in temp_edge['dstmsg']:
                dst_shape = 'oval'
            elif "'netflow': '" in temp_edge['dstmsg']:
                dst_shape = 'diamond'
            if attack_edge_flag(temp_edge['dstmsg']):
                dst_node_color = 'red'
            else:
                dst_node_color = 'blue'
            dot.node(name=str(hashgen(replace_path_name(temp_edge['dstmsg']))), label=str(
                replace_path_name(temp_edge['dstmsg']) + str(
                    partition[str(hashgen(replace_path_name(temp_edge['dstmsg'])))])), color=dst_node_color,
                     shape=dst_shape)

            if attack_edge_flag(temp_edge['srcmsg']) and attack_edge_flag(temp_edge['dstmsg']):
                edge_color = 'red'
            else:
                edge_color = 'blue'
            dot.edge(str(hashgen(replace_path_name(temp_edge['srcmsg']))),
                     str(hashgen(replace_path_name(temp_edge['dstmsg']))), label=temp_edge['edge_type'],
                     color=edge_color)

    dot.render(f'./graph_visual/subgraph_' + str(graph_index), view=False)
    graph_index += 1





  0%|                                                                                                                                                                    | 0/9 [00:00<?, ?it/s]

0.937786792946225
1.8270075086797044
thr: 3.6782980559657816


 22%|██████████████████████████████████▋                                                                                                                         | 2/9 [00:04<00:14,  2.00s/it]

2.7974665048835985
2.9327478029784757
thr: 7.196588209351312
7.006004258029685
1.0956821319431644
thr: 8.649527455944432


 33%|████████████████████████████████████████████████████                                                                                                        | 3/9 [00:44<01:57, 19.52s/it]

...

0.27373573899582565
0.7746871400413913
thr: 1.4357664490579127


 89%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                 | 8/9 [00:53<00:03,  3.74s/it]

3.123359048004586
2.351630738418813
thr: 6.650805155632805


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:56<00:00,  6.24s/it]

0.32178668871261684
0.7991852055643399
thr: 1.5205644970591268
